In [ ]:
# вимикаємо зайві попередження
import warnings
warnings.filterwarnings("ignore")

# друк всіх результатів в одній комірці а не тільки останнього
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import random
from typing import Dict
import time
from collections import deque

class SlidingWindowRateLimiter:
    """Клас для обмеження швидкості запитів користувачів за допомогою ковзаючого вікна.
    Увага!!!: цей клас не є потокобезпечним і призначений для використання в однопоточному середовищі.
    Використовується для обмеження кількості запитів від користувачів протягом певного часу.
    Не зберігає повідомлення, які були відхилені через обмеження частоти запитів (не вимагається умовами завдання).
    Тож для обробки відхилених повідомлень використовуйте інші додаткові механізми, наприклад, чергу або логування."""

    def __init__(self, window_size: int = 10, max_requests: int = 1):
        self.window_size = window_size  # розмір вікна в секундах
        self.max_requests = max_requests  # максимальна кількість запитів за вікно
        self.user_requests: Dict[str, Dict] = {} # словник для зберігання інформації про вікна користувачів
        # Ініціалізуємо словник для кожного користувача: left (час, коли вікно починається) та tickets (кількість доступних запитів)

    def _cleanup_window(self, user_id: str, current_time: float) -> None:
        """Оновлює вікно для користувача, якщо попереднє закінчилось.
        Також оновлює доступний ліміт запитів."""
        self.user_requests[user_id] = {'left': current_time, 'tickets': self.max_requests}

    def can_send_message(self, user_id: str) -> bool:
        """Перевіряє, чи може користувач надіслати повідомлення."""
        if self.user_requests.get(user_id) is None: # якщо користувача немає в словнику, то очищаємо вікно (новий користувач створюється автоматично)
            self._cleanup_window(user_id, time.time())
            return True
        elif self.user_requests[user_id]['tickets'] > 0: # якщо у користувача є доступні запити, то дозволяємо надсилання
            return True
        elif time.time() > (self.user_requests[user_id]['left'] + self.window_size): # якщо поточний час ззовні часового вікна (час початку вікна + розмір вікна),
            self._cleanup_window(user_id, time.time()) # то очищаємо вікно та дозволяємо нові запити
            return True
        else:
            return False # якщо користувач перевищив ліміт запитів або поточний час в межах вікна заборони відправки повідомлень, то не дозволяємо надсилання

    def record_message(self, user_id: str, message_id: str) -> bool:
        """Записує повідомлення користувача в історію, якщо дозволено його надіслати.
          якщо ні - повертає False, повідомлення не записується."""
        if self.can_send_message(user_id):
            message_deque.append((user_id, time.time(), message_id))
            self.user_requests[user_id]['tickets'] -= 1
            return True
        else:
            return False

    def time_until_next_allowed(self, user_id: str) -> float:
        """Повертає час до наступного дозволеного запиту для користувача."""
        if not self.can_send_message(user_id):
            return (self.user_requests[user_id]['left'] + self.window_size) - time.time()
        return 0.0

# Демонстрація роботи
def test_rate_limiter():
    # Створюємо rate limiter: вікно 10 секунд, 1 повідомлення
    limiter = SlidingWindowRateLimiter(window_size=10, max_requests=1)

    # Симулюємо потік повідомлень від користувачів (послідовні ID від 1 до 20)
    print("\n=== Симуляція потоку повідомлень ===")

    for message_id in range(1, 11):
        # Симулюємо різних користувачів (ID від 1 до 5)
        user_id = message_id % 5 + 1

        result = limiter.record_message(str(user_id), str(message_id))
        wait_time = limiter.time_until_next_allowed(str(user_id))

        print(f"Повідомлення {message_id:2d} | Користувач {user_id} | "
              f"{'✓' if result else f'× (очікування {wait_time:.1f}с)'}")

        # Невелика затримка між повідомленнями для реалістичності
        # Випадкова затримка від 0.1 до 1 секунди
        time.sleep(random.uniform(0.1, 1.0))

    # Чекаємо, поки вікно очиститься
    print("\nОчікуємо 4 секунди...")
    time.sleep(4)

    print("\n=== Нова серія повідомлень після очікування ===")

    for message_id in range(11, 21):
        user_id = message_id % 5 + 1
        result = limiter.record_message(str(user_id), str(message_id))
        wait_time = limiter.time_until_next_allowed(str(user_id))

        print(f"Повідомлення {message_id:2d} | Користувач {user_id} | "
              f"{'✓' if result else f'× (очікування {wait_time:.1f}с)'}")
        # Випадкова затримка від 0.1 до 1 секунди
        time.sleep(random.uniform(0.1, 0.5))

    print("\n=== Кінець симуляції ===")

if __name__ == "__main__":

	# черга з історією повідомлень
    message_deque = deque(maxlen=100)

    test_rate_limiter()

    print("\n=== Повідомлення в історії ===")
    for user_id, timestamp, message_id in message_deque:
        print(f"Користувач {user_id} | Час: {time.ctime(timestamp)} | Повідомлення: {message_id}")
